In [ ]:
import pandas as pd
import numpy as np

# Load Data

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("movies.csv")
print(df.columns)
df.head()

# Data Analysis

In [ ]:
# plot pairplots
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df)
plt.show()

## Rating

In [ ]:
# rating = age rating
# show rating
sns.countplot(df["rating"])
plt.show()

## Genre

In [ ]:
sns.countplot(df["genre"])
plt.show()

In [ ]:
for category in df["genre"].unique():
    df[df["genre"] == category]["score"].hist()
    plt.title(f"Genre={category}")
    plt.xlabel("score")
    plt.ylabel("frequency")
    plt.show()

In [ ]:
# print number of items in each category sorted by count
df["genre"].value_counts()

In [ ]:
print(df.country.unique().shape)
df.country.value_counts()

In [ ]:
# plot histogram for each continuous variable
for col in df.columns:
    if df[col].dtype != "object":
        df[col].hist()
        plt.title(col)
        plt.show()

## Year

In [ ]:
df["year"].hist()
plt.title("year")
plt.show()

## Released

In [ ]:
# convert column "released" to string
df["released"] = df["released"].astype(str)

In [ ]:
# how often is "year" in "released" column? str(df["year"]) in df["released"]
df["same_year"] = df.apply(lambda row: str(row["year"]) in row["released"], axis=1)
df["same_year"].value_counts()

In [ ]:
df[df["same_year"] == False].head()

## Director

In [ ]:
# do plot count of top 100 directors
df["director"].value_counts().head(20).plot(kind="bar")
plt.show()

In [ ]:
df["director"].describe()

## Writer

In [ ]:
df["writer"].value_counts().head(20).plot(kind="bar")
plt.show()

In [ ]:
df["writer"].describe()

## Star

In [ ]:
df["star"].value_counts().head(20).plot(kind="bar")
plt.show()

In [ ]:
df["star"].describe()

## Country

In [ ]:
df["country"].value_counts().head(20).plot(kind="bar")
plt.show()

In [ ]:
df["country"].describe()

## Budget

In [ ]:
df["budget"].hist()
plt.show()

In [ ]:
np.log(df["budget"]).hist()
plt.show()

## Company

In [ ]:
df["company"].value_counts().head(20).plot(kind="bar")
plt.show()

In [ ]:
df["company"].describe()

## Runtime

In [ ]:
df["runtime"].hist(bins=50)
plt.show()

In [ ]:
df.runtime.describe()

In [ ]:
df[df["runtime"] < 160]["runtime"].hist()
plt.show()

In [ ]:
np.log(df["runtime"]).hist(bins=50)
plt.show()

## Score

In [ ]:
df.score.hist()

# Feature Selection

In [ ]:
columns_to_dummies = []
columns_ignore = ["name", "released", "score", "votes", "gross", "same_year"]
df.head()

## Name

In [ ]:
df["name_len"] = df["name"].apply(len)

In [ ]:
import re

# Mapping of number words to their numeric counterparts
number_words = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
    'ten': '10'
    # Add more as needed
}


def has_number(movie):
    # Check for numeric numbers
    if re.search(r'\d', movie):
        return True

    # Check for Roman numerals
    elif re.search(r'\b[i,v,x,l,c,d,m]+\b', movie, re.I):
        return True

    # Check for number words
    else:
        for word in movie.split():
            if word.lower() in number_words:
                return True

    return False

df["has_number"] = df["name"].apply(has_number)

In [ ]:
df.iloc[[8]]

## Rating

In [ ]:
columns_to_dummies.append("rating")

## Genre

In [ ]:
columns_to_dummies.append("genre")

## Year

In [ ]:
year_min = df["year"].min()
df["year_relative"] = df["year"] - year_min

## Released

## Director

In [ ]:
# director

# keep only top 20 directors, all else will be unknown
top_directors = df["director"].value_counts().index[:20]
df.loc[~df["director"].isin(top_directors), "director"] = "Unknown"

In [ ]:
columns_to_dummies.append("director")

## Writer

In [ ]:
# writer

# keep only top 20 writers, all else will be unknown
top_writers = df["writer"].value_counts().index[:20]
df.loc[~df["writer"].isin(top_writers), "writer"] = "Unknown"

In [ ]:
columns_to_dummies.append("writer")

## Star

In [ ]:
# star

# keep only top 20 writers, all else will be unknown
top_stars = df["star"].value_counts().index[:20]
df.loc[~df["star"].isin(top_stars), "star"] = "Unknown"

In [ ]:
columns_to_dummies.append("star")

## Country

In [ ]:
columns_to_dummies.append("country")

## Budget

In [ ]:
# replace budget nan with median
df["budget"].fillna(df["budget"].median(), inplace=True)

In [ ]:
# budget
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

# Fit and transform data
df["robust_budget"] = scaler.fit_transform(df[["budget"]])
df["log_budget"] = np.log(df["budget"])


def plot_histogram(data_feature, title, x_label, y_label):
    plt.hist(data_feature, bins=60, color="blue", alpha=0.7)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(True)
    plt.show()


plot_histogram(df["budget"], "Budget Histogram", "Budget", "Count")
plot_histogram(df["robust_budget"], "Robust Budget Histogram", "Robust Budget", "Count")
plot_histogram(df["log_budget"], "Log Budget Histogram", "Log Budget", "Count")

## Company

In [ ]:
# company

# keeping top 30 companies
top_companies = df["company"].value_counts().index[:30]
df.loc[~df["company"].isin(top_companies), "company"] = "Unknown"

In [ ]:
columns_to_dummies.append("company")

## Runtime

In [ ]:
# runtime nan with median
df["runtime"].fillna(df["runtime"].median(), inplace=True)

In [ ]:
# runtime
df["log_runtime"] = np.log(df["runtime"] + 1)
plot_histogram(df["runtime"], "Runtime", "Normalized Runtime", "Frequency")

## Gross

Currently being ignored

In [ ]:
# gross

# Fit and transform data
df["robust_gross"] = scaler.fit_transform(df["gross"].values.reshape(-1, 1))
df["log_gross"] = np.log(df["gross"])

plot_histogram(df["gross"], "Gross Histogram", "Gross", "Count")
plot_histogram(df["robust_gross"], "Robust Gross Histogram", "Robust Gross", "Count")
plot_histogram(df["log_gross"], "Log Gross Histogram", "Log Gross", "Count")

## Create one-hot encoding

In [ ]:
columns_to_dummies

In [ ]:
set(df.columns) - set(columns_ignore + columns_to_dummies)

In [ ]:
df = pd.get_dummies(df, columns=columns_to_dummies)

In [ ]:
len(df.columns)

# Export Data

In [ ]:
len(df.columns)

In [ ]:
df.drop(columns=columns_ignore).head()

In [ ]:
# drop score nan
df.dropna(inplace=True)

In [182]:
df.to_excel("preprocessed.xlsx", index=False)

In [183]:
columns_ignore

['name', 'released', 'score', 'votes', 'gross', 'same_year']